# 🧪 Lab 3: Use an MCP Server as a Plugin in a Semantic Kernel Agent

In this lab, you'll learn how to **extend a Semantic Kernel agent** by connecting it to an **MCP server**. MCP (Model Context Protocol) allows agents to invoke external tools, services, or other agents as plugins.

## 🎯 What You'll Learn

You'll specifically:
- Connect to **your own Codebeamer MCP server** (which you built in **Lab 2**) as a tool via `MCPStreamableHttpPlugin`
- Create a **Semantic Kernel agent** powered by Azure OpenAI
- Use the MCP plugin inside the agent to access Codebeamer functionality
- Interact with the agent through a chat interface
- See the agent automatically call functions on the Codebeamer MCP server to answer questions

This lab showcases how Semantic Kernel can leverage **modular, tool-augmented AI workflows** by treating external MCP servers as powerful extensions to the agent's reasoning capabilities.

## ✅ Prerequisites

Before starting this lab:
1. **Complete Lab 2** - You need the Codebeamer MCP server running on `http://localhost:8080`
2. **Azure OpenAI credentials** - Ensure your `.env` file is configured with:
   - `AZURE_OPENAI_ENDPOINT`
   - `AZURE_OPENAI_KEY`
   - `MODEL_DEPLOYMENT_NAME`
   - `MODEL_DEPLOYMENT_API_VERSION`
3. **Start your MCP server** - Run `python3 servers/mcp_server.py` in a terminal

---

In [1]:
import os

from dotenv import load_dotenv

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.agents import AzureResponsesAgent

# Load environment variables from .env file
load_dotenv(override=True)

True

## Step 1: Import Required Libraries and Load Environment Variables

First, we'll import the necessary Semantic Kernel components and load our Azure OpenAI credentials.

In [2]:
# Check environment variables
print("AZURE_OPENAI_ENDPOINT:", os.getenv("AZURE_OPENAI_ENDPOINT"))
print("AZURE_OPENAI_KEY:", "***" if os.getenv("AZURE_OPENAI_KEY") else "Not set")
print("MODEL_DEPLOYMENT_NAME:", os.getenv("MODEL_DEPLOYMENT_NAME"))
print("MODEL_DEPLOYMENT_API_VERSION:", os.getenv("MODEL_DEPLOYMENT_API_VERSION"))

AZURE_OPENAI_ENDPOINT: https://short-hack-ai-foundry.cognitiveservices.azure.com/
AZURE_OPENAI_KEY: ***
MODEL_DEPLOYMENT_NAME: gpt-4o
MODEL_DEPLOYMENT_API_VERSION: 2025-03-01-preview


## Step 2: Connect to the Codebeamer MCP Server

Now we'll initialize and connect to the Codebeamer MCP server using the `MCPStreamableHttpPlugin` interface.

**Key points:**
- `MCPStreamableHttpPlugin` enables connections to MCP Servers through HTTP/HTTPS
- The URL should match your MCP server endpoint (default: `http://localhost:8080/mcp`)
- **Important:** Make sure the MCP server built in Lab 2 is running before executing this cell!

Run `python3 servers/mcp_server.py` in a separate terminal if you haven't already.

In [4]:
codebeamer_plugin = MCPStreamableHttpPlugin(
    name="Codebeamer",
    description="Codebeamer Plugin",
    url="http://localhost:8080/mcp",
)

# Start the connection to the MCP plugin
await codebeamer_plugin.connect()

CancelledError: 

## Step 3: Create the Azure OpenAI Client and Semantic Kernel Agent

In this step, we'll:
1. **Create an Azure OpenAI client** using our credentials
2. **Initialize a Semantic Kernel agent** (`AzureResponsesAgent`) that:
   - Uses Azure OpenAI for language understanding
   - Has access to the Codebeamer MCP plugin
   - Can automatically invoke Codebeamer tools when needed

The agent's instructions define its behavior and capabilities. Feel free to customize these instructions for your use case!

In [5]:
# Create the Azure OpenAI client using credentials from environment variables
client = AzureResponsesAgent.create_client(
    deployment_name=os.getenv("MODEL_DEPLOYMENT_NAME"),
    api_version=os.getenv("MODEL_DEPLOYMENT_API_VERSION"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
)

# Create the Semantic Kernel agent with the MCP plugin
codebeamer_agent = AzureResponsesAgent(
    ai_model_id=os.getenv("MODEL_DEPLOYMENT_NAME"),
    client=client,
    name="CodebeamerAgent",
    description="A chat bot that helps users interact with Codebeamer.",
    instructions="""
You are a helpful assistant that can interact with Codebeamer through the provided tools.
You can help users:
- List and explore projects
- View and manage trackers
- Get information about tracker items
- Read and post comments
When a user asks about Codebeamer data, use the available tools to fetch the information.
Provide clear, concise, and helpful responses.
""",
    plugins=[codebeamer_plugin],
)

print("✅ Semantic Kernel agent created successfully!")

✅ Semantic Kernel agent created successfully!


## Step 4: Chat with Your Agent

Now let's run an interactive chat loop! The agent will:
- Receive your questions
- Automatically decide when to call Codebeamer MCP tools
- Maintain conversation context across multiple messages
- Provide intelligent responses based on the data retrieved

**Try these example questions:**
- "What projects are available?"
- "Show me the trackers for project ID 12345"
- "Get items from tracker 5001"
- "What are the comments on item 5881865?"
- "Post a comment saying 'Looking good!' on item 5881865"

Type `exit` or `quit` to end the conversation.

In [6]:
# Create the thread - starts as None, will be set after first response
thread = None

print("💬 Starting chat with Codebeamer Agent...")
print("Type 'exit' or 'quit' to end the conversation.\n")

# Run the agent as a chat
while True:
    user_input = input("You: ")
    
    if user_input.lower() in ["exit", "quit"]:
        print("Goodbye! 👋")
        break

    # Get response from the agent
    response = await codebeamer_agent.get_response(
        messages=user_input,
        thread=thread,
    )

    # Print the response
    print(f"Bot: {response.content}\n")
    
    # Update the thread with the response to maintain conversation context
    thread = response.thread

# Clean up the connection
await codebeamer_plugin.close()
print("✅ Connection closed.")

💬 Starting chat with Codebeamer Agent...
Type 'exit' or 'quit' to end the conversation.

Bot: Let me retrieve all the projects in Codebeamer for you. One moment!

Goodbye! 👋


CancelledError: Cancelled via cancel scope 15caa9950

## 🎓 What You Learned

Congratulations! In this lab, you've successfully:

✅ Connected a Semantic Kernel agent to an external MCP server  
✅ Created a tool-augmented AI agent with Azure OpenAI  
✅ Enabled automatic tool selection and invocation  
✅ Built an interactive chat interface with conversation context  
✅ Integrated custom APIs (Codebeamer) into an AI workflow  

---

## 🚀 Next Steps & Exploration

### Extend Your Agent

You can now:
- **Add more MCP plugins** - Connect multiple MCP servers to create a multi-tool agent
- **Customize instructions** - Tailor the agent's behavior for specific use cases
- **Build workflows** - Chain multiple tool calls together for complex tasks
- **Add memory** - Implement persistent conversation history

### Explore Other MCP Servers

The real power comes from the flexibility of MCP plugins in Semantic Kernel. You can connect to **any tool that implements the Model Context Protocol (MCP)**.

🧰 **Explore These MCP Servers:**

- 🔗 [Official MCP Server Integrations (GitHub)](https://github.com/modelcontextprotocol/servers?tab=readme-ov-file#%EF%B8%8F-official-integrations)  
  A growing list of plugins including GitHub, Slack, Google Drive, PostgreSQL, and more.

- 📚 [10 Must-Know MCP Servers for Developers (DevShorts)](https://www.devshorts.in/p/ten-must-know-mcp-servers-for-every?utm_source=chatgpt.com)  
  A curated blog post with descriptions, commands, and usage tips.

- 🔧 [FastMCP Documentation](https://gofastmcp.com/)  
  Learn more about building your own MCP servers.

---

## 🧠 Key Concepts Recap

### MCPStreamableHttpPlugin
- Enables Semantic Kernel to connect to MCP servers via HTTP
- Handles tool discovery and invocation automatically
- Supports async operations for better performance

### AzureResponsesAgent
- A Semantic Kernel agent powered by Azure OpenAI
- Can accept multiple plugins for tool augmentation
- Maintains conversation context with threads
- Automatically decides when to invoke tools

### Tool-Augmented AI
- LLMs can be extended with external tools and APIs
- MCP provides a standardized protocol for tool integration
- Agents can reason about when and how to use tools
- This enables AI to interact with real-world systems and data

---

## 💡 Challenge Ideas

Try these extensions to deepen your understanding:

1. **Multi-Plugin Agent**: Add the Wikipedia MCP server from Lab 1 alongside Codebeamer
2. **Custom Instructions**: Modify the agent to specialize in specific Codebeamer workflows
3. **Error Handling**: Add try-catch blocks and graceful error messages
4. **Logging**: Track which tools are called and why
5. **Build Your Own**: Create a completely new MCP server and integrate it!